In [18]:
import pandas as pd
import numpy as np
import math as m
import DataHandling

In [222]:
roadId = input('Please write the Road ID: ')

Please write the Road ID: 


In [223]:
roads = DataHandling.readRoads(roadId)
bridges = DataHandling.readBridges(roadId)
widths = DataHandling.readWidths(roadId)
traffic = DataHandling.readTraffic(roadId)
roadnames = DataHandling.readBridges('').drop_duplicates('road', keep = 'first')['road'].tolist()

In [224]:
def formatWidthData(dataframe = widths):
    dataframe = dataframe.drop('width', 1)
    dataframe = dataframe.drop('roadId', 1)
    dataframe = dataframe.sort_values(by=['road','startChainage']).reset_index()
    dataframe = dataframe.drop('index', 1)
    return dataframe

In [225]:
def formatBridgeData(dataframe = bridges):
    dataframenoduplicates = dataframe.drop_duplicates('LRPName', keep = 'first')
    dataframemodified = dataframenoduplicates.ix[:,:10]
    dataframemodified = dataframemodified.drop('km', 1)
    dataframemodified = dataframemodified.drop('structureNr', 1)
    dataframemodified = dataframemodified.drop('type', 1)
    dataframemodified = dataframemodified.drop('roadName', 1)
    dataframemodified = dataframemodified.drop('length', 1)
    dataframemodified = dataframemodified.drop('name', 1)
    dataframemodified = dataframemodified.sort_values(by=['road','chainage']).reset_index()
    dataframemodified = dataframemodified.drop('index', 1)
    return dataframemodified

In [226]:
def formatTrafficData(dataframe = traffic, sideofroad = 'both'):
    dataframe = dataframe.drop('name',1)
    dataframe = dataframe.drop('start_lrp',1)
    dataframe = dataframe.drop('start_offset',1)
    dataframe = dataframe.drop('end_lrp',1)
    dataframe = dataframe.drop('end_offset',1)
    dataframe = dataframe.drop('length',1)
    dataframe = dataframe.reset_index()
    dataframe = dataframe.drop('index',1)
    dataframe.columns = dataframe.columns.str.replace('start_chainage','startChainage')
    dataframe.columns = dataframe.columns.str.replace('end_chainage','endChainage')
    dataframe = dataframe.sort_values(by = ['road','startChainage','linkNo'] ).reset_index()
    dataframe = dataframe.drop('index', 1)
    if sideofroad == 'both':
        dataframe = dataframe.groupby(['road','startChainage'],as_index=False).sum()
    #if sideofroad is 'left':
        #dataframe = dataframe[dataframe['linkNo'].str.contain('L')]
    return dataframe

In [227]:
def mergeFiles(trafficData=formatTrafficData(), bridgeData = formatBridgeData(), widthData = formatWidthData(), roadside = 'both'):
    indexesMergedData = widthData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
    indexesMergedData =indexesMergedData.tolist()
    addedNrLanes = bridgeData
    list_a = []
    for i in indexesMergedData:
        list_a.append(widthData['nrLanes'][int(i)-1])
    addedNrLanes['nrLanes']=list_a
    bridgeData = addedNrLanes
    addedTrafficData = bridgeData
    for title in trafficData.ix[:,3:].columns:
        indexesMergedData = trafficData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
        indexesMergedData =indexesMergedData.tolist()
        list_b=[]
        for i in indexesMergedData:
            list_b.append(trafficData[title][int(i)-1])
        addedTrafficData[title]=list_b
    return addedTrafficData

In [228]:
def calculateLaneTrafficDensity(dataframe = mergeFiles()):
    for title in dataframe.ix[:,5:].columns:
        dataframe[title] = dataframe[title]/dataframe['nrLanes']
    return dataframe

In [229]:
#Truck capacity is max capacity of type of trucks in tonnes
def calculateVulnerability(dataframe = calculateLaneTrafficDensity(), truckcapacity = [15,10,5], conditionlikelihood = [0.05,0.10,0.15,0.20]):
    vulbasematrix = dataframe.ix[:,:4]
    vulbasematrix.loc[vulbasematrix.condition == 'A', 'BridgeFailureLikelihood'] = conditionlikelihood[0]
    vulbasematrix.loc[vulbasematrix.condition == 'B', 'BridgeFailureLikelihood'] = conditionlikelihood[1]
    vulbasematrix.loc[vulbasematrix.condition == 'C', 'BridgeFailureLikelihood'] = conditionlikelihood[2]
    vulbasematrix.loc[vulbasematrix.condition == 'D', 'BridgeFailureLikelihood'] = conditionlikelihood[3]
    cargotitles = ['heavyTruck', 'mediumTruck', 'smallTruck']
    vulbasematrix['TrafficEconomicVulnerability'] = dataframe[cargotitles[0]]*truckcapacity[0]+dataframe[cargotitles[1]]*truckcapacity[1]+dataframe[cargotitles[2]]*truckcapacity[2]
    vulbasematrix['TotalEconomicVulnerability'] = vulbasematrix['TrafficEconomicVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    return vulbasematrix

In [230]:
def printRoadResults(dataframe = calculateVulnerability(),  roadId = roadId):
    dataframe.to_csv('vulnerabilityroad'+roadId+'.csv')


In [231]:
def calculatevulnerabilitysegment(dataframe = calculateVulnerability(), startChainage = 0, endChainage = 5000000):
    dataframe = dataframe[(dataframe['chainage'] >= startChainage) & (dataframe['chainage'] < endChainage)]
    result = dataframe.groupby('road', as_index = False).sum()
    return result